In [ ]:
import requests
from pprint import pprint
import os
import csv
import pandas as pd
import numpy as np
from itertools import chain

omdb_key = "9b9d69c4"

In [ ]:
award_winning_csv = 'newawardwinners.csv'
award_winning_df = pd.read_csv(award_winning_csv)
award_winning_df

In [ ]:
url = f"http://www.omdbapi.com/?apikey={omdb_key}&t="

In [ ]:
award_winning_df['Movie Genre'] = ""
award_winning_df['Runtime'] = ""
award_winning_df['Actors'] = ""
award_winning_df['IMDB Rating'] = ""
award_winning_df['Rotten Tomatoes Rating'] = ""
award_winning_df.rename(columns = {'entity':'Movie Title','year':'Year','category':'Award'}, inplace = True)
del award_winning_df['Unnamed: 0']
award_winning_df.head()

In [ ]:

for index, row in award_winning_df.iterrows():
    
    query_url = url + (row['Movie Title'])
    
    response_url = requests.get(query_url).json()
    
    try:
        award_winning_df.loc[index, 'Movie Genre'] = response_url['Genre']
        award_winning_df.loc[index, 'Actors'] = response_url['Actors']
        award_winning_df.loc[index, 'Runtime'] = response_url['Runtime']
        award_winning_df.loc[index, 'IMDB Rating'] = response_url['Ratings'][0]['Value']
        award_winning_df.loc[index, 'Rotten Tomatoes Rating'] = response_url['Ratings'][1]['Value']
        
    except (KeyError,IndexError):
        print(f" Problem with this film - '{row['Movie Title']}'")

In [ ]:
award_winning_df.head()

In [ ]:
award_winning_df['Actors'][0]

In [ ]:
# create actors data frame with year, movie & actors
actors_df = award_winning_df[['Year','Movie Title','Actors']]
actors_df

In [ ]:
def column_splitter(column_to_split):
    return list(chain.from_iterable(column_to_split.str.split(',')))

lengths = actors_df['Actors'].str.split(',').map(len)

actors_df = pd.DataFrame({
    'Year': np.repeat(actors_df['Year'],lengths),
    'Movie Title': np.repeat(actors_df['Movie Title'],lengths),
    'Actor': column_splitter(actors_df['Actors'])
})

actors_df.head()